In [ ]:
#if necessary

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#import libraries
import os
import ast
from collections import namedtuple

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from tqdm import tqdm
from PIL import Image

import joblib
from joblib import Parallel, delayed


import cv2
import albumentations
from albumentations.pytorch.transforms import ToTensorV2
from albumentations.core.transforms_interface import DualTransform
#from albumentations.augmentations.bbox_utils import denormalize_bbox, normalize_bbox

from sklearn.model_selection import StratifiedKFold

import glob
from numpy import asarray
import albumentations as A

import torch
from torch.utils.data import DataLoader, Dataset
import torch.utils.data as data_utils

from matplotlib import pyplot as plt
import matplotlib.patches as patches
from matplotlib.image import imsave

functions:


In [ ]:
def draw_rect_with_labels(img, bboxes,class_id, class_dict, color=None):
    img = img.copy()
    bboxes = bboxes[:, :4]
    bboxes = bboxes.reshape(-1, 4)
    for bbox, label in zip(bboxes, class_id):
        pt1, pt2 = (bbox[0], bbox[1]), (bbox[2], bbox[3])
        pt1 = int(pt1[0]), int(pt1[1])
        pt2 = int(pt2[0]), int(pt2[1])
        class_name = class_dict[label[4]]
        ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)
        img = cv2.rectangle(img.copy(), pt1, pt2, color, int(max(img.shape[:2]) / 200))
        img = cv2.putText(img.copy(), class_name, (int(bbox[0]), int(bbox[1]) - int(0.3 * text_height)), cv2.FONT_HERSHEY_SIMPLEX,fontScale=1,color = (255,255,255), lineType=cv2.LINE_AA)
    return img




## using my good old plotting functions
def plot_multiple_img(img_matrix_list, title_list, ncols, main_title=""):
    fig, myaxes = plt.subplots(figsize=(20, 15), nrows=1, ncols=ncols, squeeze=False)
    fig.suptitle(main_title, fontsize = 30)
    fig.subplots_adjust(wspace=0.3)
    fig.subplots_adjust(hspace=0.3)
    for i, (img, title) in enumerate(zip(img_matrix_list, title_list)):
        myaxes[i // ncols][i % ncols].imshow(img)
        myaxes[i // ncols][i % ncols].set_title(title, fontsize=15)
    plt.show()

In [ ]:
def build_df_for_image(name_img):
  image_folder_path = "../train"
  chosen_image = cv2.imread(os.path.join(image_folder_path, name_img))

  plt.imshow(chosen_image)
  chosen_image_dataframe = train_df.loc[train_df["image"]==name_img,["xmin","ymin","xmax","ymax","label"]]
  bbox_array_of_chosen_image = np.array(chosen_image_dataframe.values.tolist())

  labels_of_chosen_image = (len(bbox_array_of_chosen_image))
  print(name_img)
  print("build_df_for_image")
  print(bbox_array_of_chosen_image.shape)
  return chosen_image, bbox_array_of_chosen_image, bbox_array_of_chosen_image

In [ ]:
def draw_rect(image, bbox):
    # Draw a rectangle based on the data from the bbox
    for i in bbox:
        image_with_rect = cv2.rectangle(image, (int(i[0]), int(i[1])), (int(i[2]), int(i[3])), (0, 255, 0), 2)
        plt.imshow(image_with_rect)
    return image_with_rect

def draw_labels(image,bbox_array_of_chosen_image):
   print("draw_labels")
   draw_chosen_image = draw_rect(image.copy(), bbox_array_of_chosen_image.copy())
   plt.imshow(draw_chosen_image)




In [ ]:
def GFlip(name_img,chosen_image,bbox_array_of_chosen_image,labels_of_chosen_image):# Augmentation: Changing the image to grayscale
  ver_flip = A.Compose([
        A.ToGray(p=1),
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

  ver_flip_annotations = ver_flip(image=chosen_image, bboxes=bbox_array_of_chosen_image,labels=labels_of_chosen_image)
  ver_flip_annotations['bboxes'] = [list(bbox) for bbox in ver_flip_annotations['bboxes']]
  ver_flip_img = draw_rect_with_labels(img = ver_flip_annotations['image'], bboxes = np.array(ver_flip_annotations['bboxes']),
                          class_id = ver_flip_annotations['labels'], class_dict = {0: "a",1: "a"}, color=(255,0,0))


  #Saving the new image
  plt.imsave("../train_aug/"+"G"+name_img, ver_flip_annotations['image'])


#Saving the new label according to the augmentation to a csv file
  for i in ver_flip_annotations['bboxes']:
    check_df.loc[len(check_df),:]= ["G"+name_img, i[0], i[1] ,i[2] ,i[3] ,i[4] ]

    #print(new_row)
  #ptint the size of the csv file
  print(check_df.shape)

  plt.imshow(ver_flip_img)



In [ ]:
def HFlip(name_img,chosen_image,bbox_array_of_chosen_image,labels_of_chosen_image): #Augmentation: horizontal rotation
  ver_flip = A.Compose([
        A.HorizontalFlip(p=1),
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

  ver_flip_annotations = ver_flip(image=chosen_image, bboxes=bbox_array_of_chosen_image,labels=labels_of_chosen_image)
  ver_flip_annotations['bboxes'] = [list(bbox) for bbox in ver_flip_annotations['bboxes']]
  ver_flip_img = draw_rect_with_labels(img = ver_flip_annotations['image'], bboxes = np.array(ver_flip_annotations['bboxes']),
                          class_id = ver_flip_annotations['labels'], class_dict = {0: "a",1: "a"}, color=(255,0,0))

  #Saving the new image
  plt.imsave("../train_aug/"+"H"+name_img, ver_flip_annotations['image'])

  #Saving the new label according to the augmentation to a csv file
  for i in ver_flip_annotations['bboxes']:
    check_df.loc[len(check_df),:]= ["H"+name_img, i[0], i[1] ,i[2] ,i[3] ,i[4] ]

    #print(new_row)
  #ptint the size of the csv file
  print(check_df.shape)
  plt.imshow(ver_flip_img)


In [ ]:
def VFlip(name_img,chosen_image,bbox_array_of_chosen_image,labels_of_chosen_image): #Augmentation: vertical rotation
  ver_flip = A.Compose([
         A.VerticalFlip(p=1),
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

  ver_flip_annotations = ver_flip(image=chosen_image, bboxes=bbox_array_of_chosen_image,labels=labels_of_chosen_image)
  ver_flip_annotations['bboxes'] = [list(bbox) for bbox in ver_flip_annotations['bboxes']]
  ver_flip_img = draw_rect_with_labels(img = ver_flip_annotations['image'], bboxes = np.array(ver_flip_annotations['bboxes']),
                          class_id = ver_flip_annotations['labels'], class_dict = {1: "a"}, color=(255,0,0))

  #Saving the new image
  plt.imsave("../train_aug/"+"V"+name_img, ver_flip_annotations['image'])

  #Saving the new label according to the augmentation to a csv file
  for i in ver_flip_annotations['bboxes']:
    check_df.loc[len(check_df),:]= ["V"+name_img, i[0], i[1] ,i[2] ,i[3] ,i[4] ]

    #print(new_row)
  #ptint the size of the csv file
  print(check_df.shape)
  plt.imshow(ver_flip_img)


Main:

In [ ]:
#Preprocessing of the label csv file

train_df=pd.read_csv("../all_data.csv")
train_df['label'] = train_df['label'].replace("c",1)
train_df['image'] = train_df['image'].str[:-4] #
train_df.to_csv("../all_data.csv")
train_df

,image,xmin,ymin,xmax,ymax,label
0,Box_1_Control_100_4,0.000000,20.094567,68.114666,92.198582,1
1,Box_1_Control_100_4,36.199773,0.000000,125.443262,75.650127,1
2,Box_1_Control_100_4,53.930269,65.011829,137.854610,138.297872,1
3,Box_1_Control_100_4,6.057929,103.427901,66.932624,222.813248,1
4,Box_1_Control_100_4,56.885361,130.023645,111.258865,206.855801,1
...,...,...,...,...,...,...
718,Box_3_Acidic_100_4,149.083942,277.777787,335.254155,343.971631,1
719,Box_3_Acidic_100_4,298.611147,45.508283,382.535461,151.891262,1
720,Box_3_Acidic_100_4,196.956283,5.910170,347.074468,184.397163,1
721,Box_3_Acidic_100_4,300.975177,263.593390,480.644226,330.378269,1


In [ ]:
# Build an empty csv file for the new labels
check_df=train_df
#check_df.drop(['Unnamed: 0'], axis=1, inplace=True)
check_df = check_df.drop(check_df[check_df['label'] == 1].index)
check_df

,image,xmin,ymin,xmax,ymax,label


In [ ]:
#Going over each file in the folder and making augmentations

for ImP in glob.glob("../all_data/train/*.png"):
  pic = Image.open(ImP)
  name=str(ImP)
  #choose the pic_name according to the length of our file name
  ##control-
  if "Control" in name:
    pic_name=name[-23:]
  ##acidic
  if "Acidic" in name:
    pic_name=name[-22:]

  #print(pic_name)

  #Preprocessing of the image
  numpydata = asarray(pic)
  pic1 = Image.fromarray(numpydata)
  first_img, bbox_img , lable_img =build_df_for_image(pic_name)
  draw_labels(first_img,bbox_img)

  GFlip(pic_name,first_img,bbox_img,lable_img)
  HFlip(pic_name,first_img,bbox_img,lable_img)
  VFlip(pic_name,first_img,bbox_img,lable_img)

Save the CSV file of the label of all new augmentations

In [ ]:
check_df

In [ ]:
check_df.to_csv("../AugAll/all_lable.csv")


In [ ]:
check_df.to_json("../AugAll/all_lable.json")

In [ ]:
check_df.to_xml("../AugAll/all_lable.xml")